In [1]:
import sys
import re
import time
import datetime
# Requires for ipython to pick up on twitter?
sys.path.append('/Library/Python/2.7/site-packages/')
import twitter
import pandas as pd
import numpy as np
import func
# import pyowm # Historical API is paid

# inline plot
%matplotlib inline

/Users/albarron/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Just double checking that the below lambda will work for identifying northbound vs. southbound
#247%2

In [3]:
#%load 'data/raw-twt2016-01-26-14/21/09.csv'
df = pd.read_csv("data/all_stops_in_pa.csv",sep=',',error_bad_lines=False)
df["t_northbound"] = df.apply(lambda x:x['train_id']%2, axis=1)

In [4]:
df.head()

,train_id,time_pa,stop_pa,t_northbound
0,102,05:53,1,0
1,104,06:23,1,0
2,206,06:59,1,0
3,208,07:20,1,0
4,210,07:28,1,0


In [5]:
twts = pd.read_csv("data/formated_twts.csv",sep=',',error_bad_lines=False)

In [6]:
twts['created_at'].describe()

count                     536
unique                    475
top       2015-10-01 23:36:24
freq                        8
Name: created_at, dtype: object

In [7]:
# This is probably a really dumb approach...
from time import strptime
max_date = datetime.datetime(*strptime(twts['created_at'].max(),"%Y-%m-%d %H:%M:%S")[0:6])
min_date = datetime.datetime(*strptime(twts['created_at'].min(),"%Y-%m-%d %H:%M:%S")[0:6])
print "Max:",max_date
print "Min:",min_date

Max: 2016-01-25 23:42:14
Min: 2015-06-08 16:57:27


In [8]:
# How many days of tweet data do I have?
(max_date-min_date).days

231

In [9]:
# Stolen here: http://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

In [10]:
new_dates = df.apply(lambda x:'2015-06-23' + ' ' + x['time_pa'],axis=1)

In [11]:
example = df[['train_id','stop_pa','t_northbound']]
example['time_pa'] = pd.to_datetime(new_dates,format="%Y-%m-%d %H:%M")


In [12]:
all_trains = pd.DataFrame(columns=['train_id','stop_pa','t_northbound','time_pa'])
for cursor_day in daterange(min_date, max_date):
    this_date_series = df.apply(lambda x:cursor_day.strftime("%Y-%m-%d") + ' ' + x['time_pa'],axis=1)
    these_trains_tho = df[['train_id','stop_pa','t_northbound']]
    these_trains_tho['time_pa'] =  pd.to_datetime(this_date_series,format="%Y-%m-%d %H:%M")
    all_trains = all_trains.append(these_trains_tho)

In [14]:
all_trains.head(5)

,train_id,stop_pa,t_northbound,time_pa
0,102,1,0,2015-06-08 05:53:00
1,104,1,0,2015-06-08 06:23:00
2,206,1,0,2015-06-08 06:59:00
3,208,1,0,2015-06-08 07:20:00
4,210,1,0,2015-06-08 07:28:00


In [15]:
keys = pd.read_csv('keys.csv') # hidden from github
forecaseiokey=keys.iloc[5].string

In [16]:
import forecastio
api_key = forecaseiokey
lat = 37.441879
lng = -122.143021
#forecast = forecastio.load_forecast(api_key, lat, lng)

In [27]:
def get_avg_temp(lat,lon,timestamp):
    time = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    forecast = forecastio.load_forecast(api_key, lat, lng,time=time)

    iter = [0]*6
    temp = precip = vis = windspeed = humidity = cloudcover = 0
    for t in forecast.hourly().data:
        try:
            temp += t.d['apparentTemperature']
            iter[0]+=1
        except:
            print "No temp"
        try:
            precip += t.d['precipIntensity']
            iter[1]+=1
        except:
            print "no precip"
        try:
            vis += t.d['visibility']
            iter[2]+=1
        except:
            print "no Vis"
        try:
            windspeed += t.d['windSpeed']
            iter[3]+=1
        except:
             print "no Wind"
        try:
            humidity += t.d['humidity']
            iter[4]+=1
        except:
             print "no Humid"
        try:
            cloudcover += t.d['cloudCover']
            iter[5]+=1
        except:
            print "no Cloud"

    return pd.Series([
        timestamp,
        np.float64(temp)/np.float64(iter[0]),
        np.float64(precip)/np.float64(iter[1]),
        np.float64(vis)/np.float64(iter[2]),
        np.float64(windspeed)/np.float64(iter[3]),
        np.float64(humidity)/np.float64(iter[4]),
        np.float64(cloudcover)/np.float64(iter[5])
    ])

In [28]:
get_avg_temp(lat,lng,"2016-01-24 16:16:00")

no Cloud
no Cloud


0    2016-01-24 16:16:00
1                51.5958
2                      0
3                9.98708
4                4.24042
5               0.780833
6               0.128636
dtype: object

In [19]:
all_trains_temp = all_trains

In [22]:
all_trains_temp = all_trains_temp.set_index('time_pa')

In [29]:
temp = all_trains['time_pa'][0:100].apply(lambda x:get_avg_temp(lat,lng,str(x)))

no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud


In [30]:
temp

,0,1,2,3,4,5,6
0,2015-06-08 05:53:00,72.978750,0,10,6.681250,0.628750,0.09875
1,2015-06-08 06:23:00,72.978750,0,10,6.681250,0.628750,0.09875
2,2015-06-08 06:59:00,72.978750,0,10,6.681250,0.628750,0.09875
3,2015-06-08 07:20:00,72.978750,0,10,6.681250,0.628750,0.09875
4,2015-06-08 07:28:00,72.978750,0,10,6.681250,0.628750,0.09875
5,2015-06-08 07:38:00,72.978750,0,10,6.681250,0.628750,0.09875
6,2015-06-08 07:53:00,72.978750,0,10,6.681250,0.628750,0.09875
7,2015-06-08 08:05:00,72.978750,0,10,6.681250,0.628750,0.09875
8,2015-06-08 08:20:00,72.978750,0,10,6.681250,0.628750,0.09875
9,2015-06-08 08:28:00,72.978750,0,10,6.681250,0.628750,0.09875


In [34]:
all_train_weathers = all_trains['time_pa'].apply(lambda x:get_avg_temp(lat,lng,str(x)))

no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
no Cloud
n

In [36]:
filename = "./data/weather-trains{date}.csv".format(date="_"+str(min_date) + str(max_date))
all_train_weathers.to_csv(filename, sep='\t', encoding='utf-8')

In [38]:
all_train_weathers

,0,1,2,3,4,5,6
0,2015-06-08 05:53:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
1,2015-06-08 06:23:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
2,2015-06-08 06:59:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
3,2015-06-08 07:20:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
4,2015-06-08 07:28:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
5,2015-06-08 07:38:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
6,2015-06-08 07:53:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
7,2015-06-08 08:05:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
8,2015-06-08 08:20:00,72.978750,0,10.000000,6.681250,0.628750,0.098750
9,2015-06-08 08:28:00,72.978750,0,10.000000,6.681250,0.628750,0.098750


In [42]:
atw = all_train_weathers.rename(columns={ 0:'timestamp',
 1:'temp',
 2:'precipiation',
 3:'visability',
 4:'windspeed',
 5:'humidity',
 6:'cloudcover'})

In [43]:
atw.head()

,timestamp,temp,precipiation,visability,windspeed,humidity,cloudcover
0,2015-06-08 05:53:00,72.97875,0,10,6.68125,0.62875,0.09875
1,2015-06-08 06:23:00,72.97875,0,10,6.68125,0.62875,0.09875
2,2015-06-08 06:59:00,72.97875,0,10,6.68125,0.62875,0.09875
3,2015-06-08 07:20:00,72.97875,0,10,6.68125,0.62875,0.09875
4,2015-06-08 07:28:00,72.97875,0,10,6.68125,0.62875,0.09875


In [48]:
merged = pd.concat([all_trains, atw], axis=1)
print merged.shape
merged.head(5)

(21021, 11)


,train_id,stop_pa,t_northbound,time_pa,timestamp,temp,precipiation,visability,windspeed,humidity,cloudcover
0,102,1,0,2015-06-08 05:53:00,2015-06-08 05:53:00,72.97875,0,10,6.68125,0.62875,0.09875
1,104,1,0,2015-06-08 06:23:00,2015-06-08 06:23:00,72.97875,0,10,6.68125,0.62875,0.09875
2,206,1,0,2015-06-08 06:59:00,2015-06-08 06:59:00,72.97875,0,10,6.68125,0.62875,0.09875
3,208,1,0,2015-06-08 07:20:00,2015-06-08 07:20:00,72.97875,0,10,6.68125,0.62875,0.09875
4,210,1,0,2015-06-08 07:28:00,2015-06-08 07:28:00,72.97875,0,10,6.68125,0.62875,0.09875


In [ ]:
del merged['Unnamed: 0']

In [ ]:
merged.head()

In [51]:
filename = "./data/weather-trains{date}.csv".format(date="_"+str(min_date) + str(max_date))
merged.to_csv(filename, sep='\t', encoding='utf-8')
print filename

./data/weather-trains_2015-06-08 16:57:272016-01-25 23:42:14.csv
